# Cooling with recycling

Contents
1. **General model**

    1.1. Building characteristics and auxiliary loads
    
    1.2. Create air handing unit (AHU) object
    
2. **System without reaheating ($K_w = 0$)**

    2.1. CAV Constant Air Volume
   
    2.2. VAV Variable Air Volume : Control of supply air temperature and of of indoor air humidity by supply air mass flow rate

    2.3. Variable by-pass of the cooling coil: CAV with control of the supply air temperature and of the indoor air humidity
        
3. **System with reheating**

    3.1. CAV: control indoor temperature & humidity

    3.2. VAV: control indoor temperature & humidity and supply temperature

--------------------

Model: 
- Mixing (recirculating), cooling coil with by-pass, reheating, thermal zone, building. 
- CAV (constant air volume): the mass flow rate of supply air is constant.
- VAV (variable air volume): the mass flow rate of supply air is variable.
- VBP (variable by-pass): variable by-pass of cooling coil with constant supply air the mass flow rate. 

Control:
1. Indoor temperature controlled by the cooling-coil total heat flow rate (the supply air mass flow rate is given). Indoor humidity is not controlled.
2. Indoor temperature controlled by cooling coil; supply air temperature controlled by mass flow rate.
3. Indoor temperature controlled by cooling coil; indoor air humidity controlled by the mass flow rate.
4. Indoor temperature controlled by cooling coil; idoor air umidity controlled by the cooling-coil by-pass.

In [1]:
import ipywidgets as wd
import matplotlib.pyplot as plt
# %matplotlib inline  # uncomment for inline figure
# %matplotlib qt      # uncomment for figure in separate window
# plt.show()

plt.rcParams["figure.figsize"] = (10, 7.7)
font = {'size': 16}
plt.rc('font', **font)

In [2]:
import cool as cc

## General model

### Building characteristics and auxiliary loads

Total:
- sensible: 45 kW
- latent: 15 kW

[Rules of thumb (BSRIA 2003)](https://www.google.com/url?sa=t&rct=j&q=&esrc=s&source=web&cd=&ved=2ahUKEwi769yygc77AhXeUKQEHfobD3UQFnoECCoQAQ&url=https%3A%2F%2Fwww.bsria.com%2Fdoc%2FrQV5xn&usg=AOvVaw0sFqmRJg3NCmSeNieVawRh):
- Solar gains over floor area 150-250 W/m2 of glass (Tab. 3/p 13)
- Heat loads 50 - 100 W/m2 floor area
- U-wall 0.4 W/m2K
- U-window 3.5 W/m2K

In [3]:
import psychro as psy
# constants
c = 1e3         # J/kg K, air specific heat
l = 2496e3      # J/kg, latent heat
ρ = 1.2         # kg/m3, density

# Building dimensions
length = 20     # m
width = 30      # m
height = 3.5    # m
persons = 100   # m

sens_heat_person = 60       # W / person
latent_heat_person = 40     # W / person
load_m2 = 15        # W/m2
solar_m2 = 150      # W/m2 of window area
ACH = 1             # 1/h, air changes per Hour
U_wall = 0.4        # W/K, overall heat transfer coeff. walls
U_window = 3.5      # W/K, overall heat transfer coeff. windows

θo, φo = 32, 0.5    # outdoor temperature & relative humidity
θI, φI = 26, 0.5    # indoor temperature & relative humidity
wo = psy.w(θo, φo)
wI = psy.w(θI, φI)

floor_area = length * width
surface_floor = 2 * (length + width) * height + floor_area
surface_wall = 0.9 * surface_floor
surface_window = surface_floor - surface_wall

UA = U_wall * surface_wall + U_window * surface_window
mi = ACH * surface_floor * height / 3600 * ρ

solar_gains = solar_m2 * surface_window
electrical_load = load_m2 * surface_floor
Qsa = persons * sens_heat_person + solar_gains + electrical_load
Qla = persons * latent_heat_person

QsTZ = (UA + mi * c) * (θo - θI) + Qsa
QlTZ = mi * l * (wo - wI) + Qla

θS = θI - 15        # °C supply air temperature
m = QsTZ / c / (θI - θS)

print(f'QsTZ = {QsTZ:.0f} W, QlTZ = {QlTZ:.0f} W')
print(f'UA = {UA:.0f} W/K, mi = {mi:.2f} kg/s,\
      Qsa = {Qsa:.0f} W, Qla = {Qla:.0f} W')
print(f'm = {m:.3f} kg/s')

QsTZ = 45197 W, QlTZ = 16338 W
UA = 674 W/K, mi = 1.11 kg/s,      Qsa = 34500 W, Qla = 4000 W
m = 3.013 kg/s


### Create air handing unit (AHU) object

In [4]:
Kθ, Kw = 1e10, 0     # Kw can be 0
β = 0.2              # by-pass factor

m, mo = 3.1, 1.      # kg/s, mass flow rate: supply & outdoor (fresh) air
θo, φo = 32., 0.5    # outdoor conditions
θIsp, φIsp = 26., 0.5        # set point for indoor condition

mi = 1.35            # kg/s, mass flow rate of infiltration air
UA = 675.            # W/K, overall heat coefficient of the building
Qsa, Qla = 34000., 4000.     # W, auxiliary loads: sensible & latent

parameters = m, mo, β, Kθ, Kw
inputs = θo, φo, θIsp, φIsp, mi, UA, Qsa, Qla

# System without reaheating

To model the system without reheating, the gain of the controller for the heating coil HC is set to zero, $K_w = 0$. Therefore, the humidity in the thermal zone TZ is not controlled.

## Constant Air Volume (CAV)

![cool_CAV](./Figures/cool_CAV.svg)

> Figure 1. Constant air volume (CAV) cooling system without reheating (the sensible heat flow rate of the heating coil HC is zero, $\dot{Q}_{sHC}=0$; the indoor humidity $\varphi_4$ is not controlled, $K_w = 0$). 
> - Controlled variable: indoor air temperature $\theta_5 ≡ \theta_{I,sp}$. 
> - Command on cooling coil CC.

> Table 1. Widgets to modify the operation of the HVAC system.

|Widget| Symbol         | Unit| Meaning                               |
|------|----------------|-----|---------------------------------------|
|`θO`  |$\theta_O$      | °C  |Outdoor air temperature                |
|`φO`  |$\varphi_O$     |     |Outdoor air relative humidity          |
|`θIsp`|$\theta_I^{sp}$ | °C  |Set-point for indoor air temperature  |
|`φIsp`|$\varphi_I^{sp}$|     |Set-point for indoor relative humidity|
|`mi`  |$\dot{m}_i$     | kg/s|Mass flow rate of air infiltration into the thermal zone|
|`UA`  |$U A$           | W/°C|Overall thermal conductance of the thermal zone|
|`QsBL`|$\dot{Q}_{s,BL}$ | W   |Sensible heat of the buildng|
|`QlBL`|$\dot{Q}_{l,BL}$ | W   |Latent heat of the building|

In [5]:
cool1 = cc.MxCcRhTzBl(parameters, inputs)
wd.interact(cool1.CAV_wd, θo=(26, 34), φo=(0.4, 1), θIsp=(20, 28), φIsp=(0.4, 1),
            mi=(0.5, 3, 0.1), UA=(500, 800, 10), QsBL=(0, 60_000, 500), QlBL=(0, 20_000, 500));

interactive(children=(IntSlider(value=32, description='θo', max=34, min=26), FloatSlider(value=0.8, descriptio…

> Figure 2. CAV cooling without reheating. Points 3 and 4 are superposed on the pshychrometric diagram, $\theta_3 ≡ \theta_4$.
> - Controlled variable: indoor air temperature $\theta_5 ≡ \theta_{I,sp}$. 
> - Command on cooling coil CC.

## Variable Air Volume (VAV)

### Control of supply air temperature by supply air mass flow rate

![cool_VAV_θS](./Figures/cool_VAV_θS_noheat.svg)

> Figure 3. Variable air volume (VAV) cooling system without reheating.
The controller $K_w$ is disabled, $K_w = 0$; the sensible heat flow rate of the heating coil HC is zero, ($\dot{Q}_{sHC}=0$).  The indoor humidity $\varphi_4$ is not controlled. 
> - Controlled variables: supply air temperature $\theta_3 = \theta_4 ≡ \theta_S$ and indoor air temperature $\theta_4 ≡ \theta_{I,sp}$.
> - Commands on cooling coil $\dot{Q}_{tCC}$ and mass flow rate of supply air $\dot{m}_{da}$ (fan, F).

> Table 2. Widgets to modify the operation of the HVAC system.

|Widget| Symbol         | Unit| Meaning                               |
|------|----------------|-----|---------------------------------------|
|`θS`  |$\theta_S^{sp}$ | °C  |Set-point of supply air temperature    |
|`θO`  |$\theta_O$      | °C  |Outdoor air temperature                |
|`φO`  |$\varphi_O$     |     |Outdoor air relative humidity          |
|`θIsp`|$\theta_I^{sp}$ | °C  |Set-point for indoor air temperature   |
|`φIsp`|$\varphi_I^{sp}$|     |Set-point for indoor relative humidity |
|`mi`  |$\dot{m}_i$     | kg/s|Mass flow rate of air infiltration into the thermal zone|
|`UA`  |$U A$           | W/°C|Overall thermal conductance of the thermal zone|
|`QsBL`|$\dot{Q}_{s,a}$ | W   |Auxiliary sensible heat of the thermal zone|
|`QlBL`|$\dot{Q}_{l,a}$ | W   |Auxiliary latent heat of the thermal zone|


In [6]:
cool2 = cc.MxCcRhTzBl(parameters, inputs)
wd.interact(cool2.VAV_wd, value='θS', sp=(14, 21), θo=(28, 36), φo=(0.4, 1),
            θIsp=(22, 26), φIsp=(0.4, 0.8),
            mi=(0.5, 3, 0.1), UA=(500, 800, 10), QsBL=(0, 60_000, 500), QlBL=(0, 20_000, 500));

interactive(children=(Text(value='θS', description='value'), IntSlider(value=18, description='sp', max=21, min…

> Figure 4. VAV cooling without reheating. Points 3 and 4 are superposed on the pshychrometric diagram. Controlled variables: supply air temperature $\theta_3 = \theta_4 ≡ \theta_S$ and indoor air temperature $\theta_4 ≡ \theta_{I,sp}$. Commands on cooling coil CC and fan (for mass flow rate of supply air) F.

### Control of indoor air humidity by supply air mass flow rate

![cool_VAV_φI](./Figures/cool_VAV_φI.svg)

> Figure 5. Variable air volume (VAV) cooling system without reheating (the sensible heat flow rate of the heating coil HC is zero, $\dot{Q}_{sHC}=0$).  The indoor humidity $\varphi_5$ is controlled by a non-linear (least-squares) controller. Controlled variables: indoor air temperature $\theta_5$ and relative humidity $\varphi_5$.

> Table 3. Widgets to modify the operation of the HVAC system.

|Widget| Symbol         |          | Unit| Meaning                               |
|------|----------------|----------|-----|---------------------------------------|
|`φI`  |$\varphi_I^{sp}$ |         |   |Set-point of indoor air relative humidity|
|`θO`  |$\theta_O$      |$\theta_0$| °C  |Outdoor air temperature                |
|`φO`  |$\varphi_O$     |$\varphi_0$|    |Outdoor air relative humidity          |
|`θIsp`|$\theta_I^{sp}$ |$\theta_3^{sp}$| °C|Set-point for indoor air temperature  |
|`φIsp`|$\varphi_I^{sp}$|$\varphi_3^{sp}$|  |Set-point for indoor relative humidity|
|`mi`  |$\dot{m}_i$     | | kg/s|Mass flow rate of air infiltration into the thermal zone|
|`UA`   |$U A$           | | W/°C|Overall thermal conductance of the thermal zone|
|`QsBL` |$\dot{Q}_{s,a}$ | | W   |Auxiliary sensible heat of the thermal zone|
|`QlBL` |$\dot{Q}_{l,a}$ | | W   |Auxiliary latent heat of the thermal zone|


The indoor air humidity is controlled by the mass flow rate of dry air, $\dot{m}_{da}$ in the Air Handing Unit (AHU).
The heating coil HC is not used (*KwI* = 0).

In [7]:
cool3 = cc.MxCcRhTzBl(parameters, inputs)
wd.interact(cool3.VAV_wd, value='φI', sp=(0.4, 0.8), θo=(28, 36), φo=(0.4, 1),
            θIsp=(22, 26), φIsp=(0.4, 0.8),
            mi=(0.5, 3, 0.05), UA=(500, 800, 10), Qsa=(0, 60_000, 500), Qla=(0, 20_000, 500));

interactive(children=(Text(value='φI', description='value'), FloatSlider(value=0.8, description='sp', max=0.8,…

>Figure 6. VAV cooling without reheating with control of indoor temperature and humidity. Points 3 and 4 are superposed on the pshychrometric diagram.

## Variable by-pass of the cooling coil

### CAV with control of supply air temperature $\theta_S$

![Figures/cool_CAVb_θS](./Figures/cool_CAVb_θS.svg)

> Figure 7. Constant air volume (CAV) cooling system without reheating (the sensible heat flow rate of the heating coil HC is zero, $\dot{Q}_{sHC}=0$).  The supply air temperature $\theta_4$ is controlled by a non-linear (least-sqaures) controller. Controlled variable: indoor air temperature $\theta_4$.

> **Note**: The supply air temperature, $\theta_S$, cannot be controlled when $\dot{m}$, $\theta_I$ and $\dot{Q}_{sTZ}$ are imposed. This results from the sensible heat balance on the thermal zone: $\dot{m}c\theta_S - \dot{m}c\theta_I + \dot{Q}_{sTZ} = 0$


### CAV control of indoor air humidity
                         
![cool_CAVb_φI.png](./Figures/cool_CAVb_φI.svg)

> Figure 8. Constant air volume (CAV) cooling system without reheating (the sensible heat flow rate of the heating coil HC is zero, $\dot{Q}_{sHC}=0$). The indoor air humidity $\varphi_5$ is controlled by a non-linear (least-squares) controller. Controlled variable: indoor air temperature $\theta_4$.

> Table 4. Widgets to modify the operation of the HVAC system.

|Widget| Symbol         |          | Unit| Meaning                               |
|------|----------------|----------|-----|---------------------------------------|
|`φI`  |$\varphi_I^[sp]$ |         |   |Set-point of indoor air relative humidity|
|`θo`  |$\theta_O$      |$\theta_0$| °C  |Outdoor air temperature                |
|`φo`  |$\varphi_O$     |$\varphi_0$|    |Outdoor air relative humidity          |
|`θIsp`|$\theta_I^{sp}$ |$\theta_3^{sp}$| °C|Set-point for indoor air temperature  |
|`φIsp`|$\varphi_I^{sp}$|$\varphi_3^{sp}$|  |Set-point for indoor relative humidity|
|`mi`  |$\dot{m}_i$     | | kg/s|Mass flow rate of air infiltration into the thermal zone|
|`UA`   |$U A$           | | W/°C|Overall thermal conductance of the thermal zone|
|`QsBL` |$\dot{Q}_{s,a}$ | | W   |Auxiliary sensible heat of the thermal zone|
|`QlBL` |$\dot{Q}_{l,a}$ | | W   |Auxiliary latent heat of the thermal zone|

In [8]:
cool4 = cc.MxCcRhTzBl(parameters, inputs)
wd.interact(cool4.VBP_wd, value='φI', sp=(0.2, 0.6), θo=(28, 36), φo=(0.4, 1),
            θIsp=(22, 26), φIsp=(0.4, 0.8),
            mi=(0.5, 3, 0.1), UA=(500, 800, 10), Qsa=(0, 60_000, 500), Qla=(0, 20_000, 500));

interactive(children=(Text(value='φI', description='value'), FloatSlider(value=0.6, description='sp', max=0.6,…

**Note**: By changing the initial guess for $\beta$ in `β_ls(self, value, sp)`, a second solution may be found (i.e. the second secant point of the saturation curve).  

# System with reheating

## CAV: control indoor temperature & humidity

![cool_CAV](./Figures/cool_CAV.svg)

> Figure 9. Constant air volume (CAV) cooling system with reheating. The indoor air temperature $\theta_4$ and humidity $\varphi_4$ are controlled.

> Table 5. Widgets to modify the operation of the HVAC system.

|Widget| Symbol         |          | Unit| Meaning                               |
|------|----------------|----------|-----|---------------------------------------|
|`θo`  |$\theta_O$      |$\theta_0$| °C  |Outdoor air temperature                |
|`φo`  |$\varphi_O$     |$\varphi_0$|    |Outdoor air relative humidity          |
|`θIsp`|$\theta_I^{sp}$ |$\theta_3^{sp}$| °C|Set-point for indoor air temperature  |
|`φIsp`|$\varphi_I^{sp}$|$\varphi_3^{sp}$|  |Set-point for indoor relative humidity|
|`mi`  |$\dot{m}_i$     | | kg/s|Mass flow rate of air infiltration into the thermal zone|
|`UA`   |$U A$           | | W/°C|Overall thermal conductance of the thermal zone|
|`QsBL` |$\dot{Q}_{s,a}$ | | W   |Auxiliary sensible heat of the thermal zone|
|`QlBL` |$\dot{Q}_{l,a}$ | | W   |Auxiliary latent heat of the thermal zone|

In [9]:
cool5 = cc.MxCcRhTzBl(parameters, inputs)
β, Kw = 0.2, 1e10
cool5.actual[[2, 4]] = β, Kw
wd.interact(cool5.CAV_wd, θo=(26, 34), φo=(0.4, 1), θIsp=(20, 28), φIsp=(0.4, 1),
            mi=(0.5, 3, 0.1), UA=(500, 800, 10), Qsa=(0, 60_000, 500), Qla=(0, 20_000, 500));

interactive(children=(IntSlider(value=32, description='θo', max=34, min=26), FloatSlider(value=0.8, descriptio…

## VAV: control indoor temperature & humidity and supply temperature

![cool_VAV_θS](./Figures/cool_VAV_θS.svg)

> Figure 10. Variable air volume (VAV) cooling system with reheating. The indoor air temperature $\theta_4$ and humidity $\varphi_4$ and the supply air temperature $\theta_4$ are controlled.

> Table 5. Widgets to modify the operation of the HVAC system.

|Widget| Symbol         |          | Unit| Meaning                               |
|------|----------------|----------|-----|---------------------------------------|
|`θS`  |$\theta_S^[sp]$ |          | °C  |Set-point of supply air temperature    |
|`φI`  |$\varphi_I^[sp]$ |         |   |Set-point of indoor air relative humidity|
|`θo`  |$\theta_O$      |$\theta_0$| °C  |Outdoor air temperature                |
|`φo`  |$\varphi_O$     |$\varphi_0$|    |Outdoor air relative humidity          |
|`θIsp`|$\theta_I^{sp}$ |$\theta_3^{sp}$| °C|Set-point for indoor air temperature  |
|`φIsp`|$\varphi_I^{sp}$|$\varphi_3^{sp}$|  |Set-point for indoor relative humidity|
|`mi`  |$\dot{m}_i$     | | kg/s|Mass flow rate of air infiltration into the thermal zone|
|`UA`   |$U A$           | | W/°C|Overall thermal conductance of the thermal zone|
|`QsBL` |$\dot{Q}_{s,a}$ | | W   |Auxiliary sensible heat of the thermal zone|
|`QlBL` |$\dot{Q}_{l,a}$ | | W   |Auxiliary latent heat of the thermal zone|

In [10]:
cool6 = cc.MxCcRhTzBl(parameters, inputs)
Kw = 1e10
cool6.actual[4] = Kw
wd.interact(cool6.VAV_wd, value='θS', sp=(14, 21), θo=(28, 36), φo=(0.4, 1),
            θIsp=(22, 26), φIsp=(0.4, 0.8),
            mi=(0.5, 3, 0.1), UA=(500, 800, 10), Qsa=(0, 60_000, 500), Qla=(0, 20_000, 500));

interactive(children=(Text(value='θS', description='value'), IntSlider(value=18, description='sp', max=21, min…

# Bibliography

1. TRANE (2000) [Dehumidify with Constant-Volume Systems](https://www.trane.com/content/dam/Trane/Commercial/global/products-systems/education-training/engineers-newsletters/airside-design/enews_29_04_082800.pdf)

2. TRANE (2013) [Understanding Single-Zone VAV Systems](https://www.trane.com/content/dam/Trane/Commercial/global/products-systems/education-training/engineers-newsletters/airside-design/admapn047en_0413.pdf)

3. TRANE (2005) [CO2-Based Demand-Controlled Ventilation](https://www.trane.com/content/dam/Trane/Commercial/global/products-systems/education-training/engineers-newsletters/airside-design/admapn017en_1005.pdf)

4. BSRIA (2003) [Rules of thumb](https://www.google.com/url?sa=t&rct=j&q=&esrc=s&source=web&cd=&ved=2ahUKEwi769yygc77AhXeUKQEHfobD3UQFnoECCoQAQ&url=https%3A%2F%2Fwww.bsria.com%2Fdoc%2FrQV5xn&usg=AOvVaw0sFqmRJg3NCmSeNieVawRh)